In [1]:
# is a good pratice only import what you need in the notebook , ill refactor this later in these notebooks :)

import sys
sys.path.append(r'C:\Users\bield\Documents\projsData\proj-1\utils')
from utils import (createSparkSesion,
                  takePandas)
from pyspark.sql import (functions as F,
                        Window)
import matplotlib.pyplot as plt



In [2]:
sys.path.append(r'C:\Users\bield\Documents\projsData\proj-1\utils')

In [3]:
spark = createSparkSesion()

In [4]:
def rename_columns(df, dic):
    df_columns = df.columns

    for column in df_columns:
        try:
            
            new_name = dic[column]
            df = df.withColumnRenamed(column, new_name)
        except KeyError:
            if column in dic.values():
                print("column already renamed")
            else:
                print(f"column not found in dict {column} , {KeyError}")#tratando erros

    return df


    

In [5]:
#padronizando as colunas e deixando o entendimento mais facil por meio dos proprios nomes
rename_columns_dic = {
                    "ProdutoID"     :"product_id",
                    "VendasID"      :"sale_id",
                    "Quantidade"    :"quantity",
                    "ValorUnitario" :"unit_value",
                    "ValorTotal"    :"total_value",
                    "ClienteID"     :"customer_id",
                    "Cliente"       :"custome_name",
                    "Estado"        :"state",
                    "Genero"        :"gender",
                    "Status"        :"client_tier",
                    "Produto"       :"product_name",
                    "Preco"         :"price",
                    "VendedorID"    :"seller_id",
                    "Data"          :"date",
                    "Total"         :"sale_total_amount",
                    "Vendedor"      :"seller_name",
                    "Desconto"      :"discount_amount"
                    }

In [6]:
prefix = r"../../datasets/download/Atividades/"


In [7]:
df_customers   = spark.read.parquet(f"{prefix}/Clientes.parquet")
df_sales_itens = spark.read.parquet(f"{prefix}/ItensVendas.parquet")
df_products    = spark.read.parquet(f"{prefix}/Produtos.parquet")
df_sales       = spark.read.parquet(f"{prefix}/Vendas.parquet")
df_seller      = spark.read.parquet(f"{prefix}/Vendedores.parquet")

In [8]:
df_customers.takePandas(2)

,ClienteID,Cliente,Estado,Genero,Status
0,1,Adelina Buenaventura,RJ,M,Silver
1,2,Adelino Gago,RJ,M,Silver


In [9]:
df_sales_itens.takePandas(2)

,ProdutoID,VendasID,Quantidade,ValorUnitario,ValorTotal,Desconto,TotalComDesconto
0,2,400,2,9201.0,18402.0,"6256,68",12145.32
1,2,385,2,9201.0,18402.0,"5704,62",12697.38


In [10]:

df_products.takePandas(2)

,ProdutoID,Produto,Preco
0,1,Bicicleta Aro 29 Mountain Bike Endorphine 6.3 ...,"8.852,00"
1,2,Bicicleta Altools Stroll Aro 26 Freio À Disco ...,"9.201,00"


In [11]:
df_sales.takePandas(2)

,VendasID,VendedorID,ClienteID,Data,Total
0,1,1,91,1/1/2019,8053.6
1,2,6,185,1/1/2020,150.4


In [12]:
df_seller.takePandas(2)

,VendedorID,Vendedor
0,1,Armando Lago
1,2,Capitolino Bahía


In [13]:
#my objective here is not to do a profund business analysis , just a few insights to build a pyspark code , and exemplify my abilities in build a ETL , desnormazile tables etc...

In [14]:
#to do

#sales mans metrics
#with seller give more discounts
#best sellers by month >by amount , by sales
#objective  > identify best emplooyes ,for maybe a bonus , and find a possible corelation between give discounts, and be the best seller



#customers metrics 
#top customer , by amount , by recurrence
#maybe detect with customer we could offer discounts , send marketing campaings, trigger some fidelity bonus etc...




#store metrics:
#month over month
#month distribuition , by amount , by untary sales
#understando general store metrics



#product metrics:
#top product , by unitys sold, by amount
#understand which product we sell more , for some kind of storage managment , tendencies etc..



<h1>Sellers metrics 

a maneger could use this etl to track sellers performance , for example

In [15]:
df_sellers_metrics  = (
                        df_sales.join(df_seller,on="VendedorID",how="left")
                                .join(df_sales_itens,on="VendasID",how="left")
                                .select(
                                        F.expr("VendasID as sale_id"),
                                        F.expr("Vendedor as seller_name"),
                                        F.expr("Total as total_sale_amount"),
                                        F.expr("ProdutoId as product_id"),
                                        F.expr("TotalComDesconto as item_final_price_with_discont"),
                                        F.expr("Desconto as discont"),
                                        F.expr("Data as date")
                                        )
                                .withColumn("discont",
                                                        F.when(F.col("discont").rlike(r"-"),0.00)
                                                         .otherwise(
                                                                        F.regexp_replace(F.col("discont"),r",",r".")
                                                                        .cast("double")) # tratando os nulos
                                            )
                                .withColumn("date",F.to_date(F.to_timestamp(F.col("date"), "d/M/yyyy"))) # cast para time stamp facilita funcoes de calculo e operacoes com datas
                                .withColumn("month",F.month(F.col("date")))
                                .withColumn("year",F.year(F.col("date")))
                                .drop("date")
                        )

In [16]:
total_sales_value_window        = Window.partitionBy("month","year").orderBy(F.desc("total_sales_value"))
total_sales_window              = Window.partitionBy("month","year").orderBy(F.desc("total_sales"))
total_discount_given_window     = Window.partitionBy("month","year").orderBy(F.desc("total_discount_given"))

In [36]:
df_top_sellers = (df_sellers_metrics
                        .groupBy("month","year","seller_name")
                        .agg(
                            F.sum("total_sale_amount").alias("total_sales_value"),
                            F.count("sale_id").alias("total_sales"),
                            F.sum("discont").alias("total_discount_given")
                            )
                        .withColumn("rank_seller_by_amount",F.rank().over(total_sales_value_window))
                        .withColumn("rank_seller_by_sales",F.rank().over(total_sales_window))
                        .withColumn("rank_seller_by_discont",F.rank().over(total_discount_given_window))
                        .select("month","year","seller_name","total_sales_value","rank_seller_by_amount","total_sales","rank_seller_by_sales","total_discount_given","rank_seller_by_discont")#just to order column orders
                        )

In [37]:
df_top_sellers.filter((F.col("month")==1)&(F.col("year")==2020)).takePandas(20)

,month,year,seller_name,total_sales_value,rank_seller_by_amount,total_sales,rank_seller_by_sales,total_discount_given,rank_seller_by_discont
0,1,2020,Hélio Liberato,18402.00,5,1,7,184.02,1
1,1,2020,Godo Capiperibe,47467.20,1,3,4,88.52,2
2,1,2020,Jéssica Castelão,37258.58,3,11,1,78.28,3
3,1,2020,Daniel Pirajá,40878.12,2,4,2,61.54,4
4,1,2020,Iberê Lacerda,9711.88,6,3,4,24.19,5
5,1,2020,Armando Lago,526.16,7,2,6,2.63,6
6,1,2020,Capitolino Bahía,33357.08,4,4,2,0.00,7


In [19]:
# after all this ET(~L) process i could Load this date somewhere , maybe in a db where a manager could access  , schedulle this pipe on airflow etc..

<h1>Customer metrics

In [20]:
 #clients metrics 
#top client , by amount , by recurrence
#top item purchased by customer
#maybe detect with client we could offer discounts , send marketing campaings, trigger some fidelity bonus, holidays campaings etc...

Idealmente focando na performance , eu deveria desnormalizar olhando para os ids e posteriormente ja com o resultado esperado cruzar para obter os campos descritivos, para economizar memoria ram em um cenario de big data 
mas esse nao é o caso...

data > timestamp > date
desconto > tratar nulos


In [22]:
# isso deveria ter sido feito logo de primeira , so percebi posteriormente ,  mas mantive separado para demonstrar outras abordagens no codigo ...
df_sales        = rename_columns(df_sales,       rename_columns_dic)
df_sales_itens  = rename_columns(df_sales_itens, rename_columns_dic)
df_products     = rename_columns(df_products,    rename_columns_dic)
df_customers    = rename_columns(df_customers,   rename_columns_dic)

column not found in dict TotalComDesconto , <class 'KeyError'>


In [78]:
df_customers_metrics = (  df_customers.join(df_sales,on="customer_id",how="left")
                                   .join(df_sales_itens,on="sale_id",how="left")
                                   .join(df_products,on="product_id",how="left")
                                   .withColumn("date",F.to_date(F.to_timestamp(F.col("date"),"d/m/yyyy")))
                                   .withColumn("month",F.month(F.col("date")))
                                   .withColumn("year",F.year(F.col("date")))
                                   .drop("product_id","customer_id","seler_id","date")
                                   .withColumn(
                                                "discount_amount",
                                                F.when(
                                                    (F.col("discount_amount").rlike(r"-")) | (F.col("discount_amount") == ""),
                                                    0.00
                                                ).otherwise(
                                                    F.regexp_replace(F.col("discount_amount"), r",", r".").cast("double")
                                                )  # tratando os nulos
                                            )
    
                        )

In [80]:
rank_customer_by_amount_spend_window        = Window.partitionBy("month","year").orderBy(F.desc("customer_total_spend"),F.desc("customer_total_purchases"),F.desc("customer_total_discount_amount"))
rank_customer_by_total_purchases_window     = Window.partitionBy("month","year").orderBy(F.desc("customer_total_purchases"),F.desc("customer_total_spend"),F.desc("customer_total_discount_amount"))
rank_customer_by_discount_recived_window    = Window.partitionBy("month","year").orderBy(F.desc("customer_total_discount_amount"),F.desc(F.col("discount_share")))#priorizar pelo menor share de desconto

columnOrderFinal = ["year","month","customer_total_spend","rank_customer_by_amount","customer_total_purchases","rank_customer_by_total_purchases","customer_total_discount_amount","rank_customer_by_discount_recived"]

In [81]:
df_top_customers  = (df_customers_metrics
                        .groupBy("month","year","custome_name")
                        .agg(
                            F.sum(F.col("sale_total_amount")).alias("customer_total_spend"),
                            F.count(F.col("sale_id")).alias("customer_total_purchases"),
                            F.sum(F.col("discount_amount")).alias("customer_total_discount_amount")
                            
                        )
                        .withColumn("discount_share",F.expr("cast(customer_total_discount_amount as int)/customer_total_purchases"))
                        .withColumn("rank_customer_by_amount",F.rank().over(rank_customer_by_amount_spend_window))
                        .withColumn("rank_customer_by_total_purchases",F.rank().over(rank_customer_by_total_purchases_window))
                        .withColumn("rank_customer_by_discount_recived",F.rank().over(rank_customer_by_discount_recived_window))
                        .select(*columnOrderFinal)
                     )

In [84]:
df_top_customers.filter(F.col("year")==2020).takePandas(10)

,year,month,customer_total_spend,rank_customer_by_amount,customer_total_purchases,rank_customer_by_total_purchases,customer_total_discount_amount,rank_customer_by_discount_recived
0,2020,1,535900.42,1,61,1,18598.34,1
1,2020,1,290705.97,2,34,2,11938.38,2
2,2020,1,62342.32,10,5,14,5866.57,3
3,2020,1,18402.00,26,1,40,5704.62,4
4,2020,1,87541.20,6,3,24,5142.52,5
5,2020,1,26221.20,20,2,29,4851.84,6
6,2020,1,48521.60,13,4,17,4148.45,7
7,2020,1,49915.94,12,7,9,3117.38,8
8,2020,1,97127.50,5,10,4,3021.14,9
9,2020,1,31976.70,19,3,25,2906.05,10
